In [1]:
# imports
import numpy as np
import pandas as pd
import json
import requests
import os
import sys
from pathlib import Path

from covid_package.libs.store_data import read_data
from covid_package.libs.valid_keys import fetch_l0_keys, fetch_l1_keys, fetch_l2_keys
from covid_package.libs.aggregate_data import fetch_latest_data_date
from covid_package.api.get_country_records import get_country_records
from covid_package.libs.country_list import fetch_countries
from covid_package.api.get_country_data import get_level_1_data

In [2]:
# set up data file path
CURRENT_DIR = os.path.abspath('')
sys.path.append(CURRENT_DIR)

FILE_NAME = 'owid-covid-data.json'
file_path = os.path.join(CURRENT_DIR, 'data', FILE_NAME)
DATA_FILE = Path(file_path)

# read the data file from the data dir
data = read_data(DATA_FILE)

In [3]:
# populate the keys
key_list = fetch_l0_keys(data)
country_list = fetch_countries(data)
print("Latest data is:", fetch_latest_data_date(data, key_list))

Latest data is: 2021-02-28


In [17]:
# list of valid level 1 keys
#print(fetch_l1_keys(data, key_list))

In [18]:
# list of valid level 2 keys
#print(fetch_l2_keys(data, key_list))

In [20]:
# for each country, get record numbers and data dates
#print(get_country_records(data, key_list))

In [4]:
req_res = ['population', 'gdp_per_capita']
l1_data = get_level_1_data(data, key_list, req_res)

for key in l1_data:
    print(key, l1_data[key])


AFG {'population': 38928341.0, 'gdp_per_capita': 1803.987}
ALB {'population': 2877800.0, 'gdp_per_capita': 11803.431}
DZA {'population': 43851043.0, 'gdp_per_capita': 13913.839}
AND {'population': 77265.0}
AGO {'population': 32866268.0, 'gdp_per_capita': 5819.495}
AIA {'population': 15002.0}
ATG {'population': 97928.0, 'gdp_per_capita': 21490.943}
ARG {'population': 45195777.0, 'gdp_per_capita': 18933.907}
ARM {'population': 2963234.0, 'gdp_per_capita': 8787.58}
AUS {'population': 25499881.0, 'gdp_per_capita': 44648.71}
AUT {'population': 9006400.0, 'gdp_per_capita': 45436.686}
AZE {'population': 10139175.0, 'gdp_per_capita': 15847.419}
BHS {'population': 393248.0, 'gdp_per_capita': 27717.847}
BHR {'population': 1701583.0, 'gdp_per_capita': 43290.705}
BGD {'population': 164689383.0, 'gdp_per_capita': 3523.984}
BRB {'population': 287371.0, 'gdp_per_capita': 16978.068}
BLR {'population': 9449321.0, 'gdp_per_capita': 17167.967}
BEL {'population': 11589616.0, 'gdp_per_capita': 42658.576}
B